# Transforming graph data to pandas dataframe

To make a pandas dataframe from the graphdata it is first necessary to import the data. How to do this is explained on: <br> https://github.com/alkem-io/analytics-playground 

In [2]:
%run Functions.ipynb

To use some of the funtions used in this notebook you first need to download the Functions notebook, which is available in jupyter notebook and spyder.

In [3]:
import pandas as pd
import json
from pandas import json_normalize

When you have downloaded the matrix as JSON-format, which is used to create the graph. With open you can open the data to make it as a pandas dataframe. <br><br>
Within the variable_name['nodes'] it is possible to reach the information of the hubs, challenges, opportunities and contributors

In [4]:
with open('transformed-graph-data-new.json', 'r', encoding='utf-8') as file:
    variable_name = json.load(file)

### Node information

Get all the node information of the hubs, contributors, challenges and opportunities

In [6]:
nodes = variable_name['nodes']

In [26]:
hubs = json_normalize(nodes['hubs'])

In [10]:
contributors = json_normalize(nodes['contributors'])

In [30]:
challenges = json_normalize(nodes['challenges'])

In [34]:
opportunities = json_normalize(nodes['opportunities'])

Save the hub, challenge, opportunity and contributors as csv so you can reach them any time necessary

In [22]:
hubs.to_csv('graph_hubs_2023')
challenges.to_csv('graph_challenges_2023')
opportunities.to_csv('graph_opportunities_2023')
contributors.to_csv('graph_contributors_2023')

In [23]:
print('column available for the hubs, challenges, opportunities and contributors are:')
for i in hubs.columns:
    print(i)

column available for the hubs, challenges, opportunities and contributors are:
id
nameID
displayName
type
group
weight
url
avatar
country
city
lon
lat
leadOrgsCount


### Edge information

In [25]:
edges = json_normalize(variable_name['edges'])

In [26]:
print('column available for the edges are:')
for i in edges.columns:
    print(i)

column available for the edges are:
sourceID
targetID
source
target
weight
type
group


The sourceID = source, and targetID=target

## Connect sourceID with node names
the goal for creating this dataframe is to make a dataframe which shows in the rows the different users and in the journeys the communities. The value is 1 if a user is a member of the journey. The only infromation that is kept of the hubs, challenges, opportunity and contributors are the id, name, type and group

In [28]:
hubs_min = hubs.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat', 'leadOrgsCount'], axis=1)
challenges_min = challenges.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat', 'leadOrgsCount'], axis=1)
opportunities_min = opportunities.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat', 'leadOrgsCount'], axis=1)
users_min = contributors.drop(['nameID','weight','url', 'avatar', 'country', 'city', 'lon', 'lat'], axis=1)

### Connect Hubs

In [29]:
merge = pd.merge(edge_min, hubs_min, left_on='source', right_on='id', how='left')

In [31]:
merge = merge.drop(['id'], axis=1)

In [32]:
merge = merge.rename(columns={'displayName':'name Source', 'type_x':'type Edge', 'type_y':'type Source', 
                              'group_y':'group Source', 'group_x':'group Edge'})

### Connect Challenges

In [34]:
merge2 = pd.merge(merge, challenges_min, left_on='source', right_on='id', how='left')

In [36]:
merge2['name Source'] = merge2['name Source'].combine_first(merge2['displayName'])
merge2['type Source'] = merge2['type Source'].combine_first(merge2['type'])
merge2['group Source'] = merge2['group Source'].combine_first(merge2['group'])

In [37]:
merge2 = merge2.drop(['id', 'displayName', 'type', 'group'], axis=1)

### Connect Opportunities

In [39]:
merge3 = pd.merge(merge2, opportunities_min, left_on='source', right_on='id', how='left')

In [41]:
merge3['name Source'] = merge3['name Source'].combine_first(merge3['displayName'])
merge3['type Source'] = merge3['type Source'].combine_first(merge3['type'])
merge3['group Source'] = merge3['group Source'].combine_first(merge3['group'])

In [42]:
merge3 = merge3.drop(['id', 'displayName', 'type', 'group'], axis=1)

### Connect Contributors

In [44]:
merge4 = pd.merge(merge3, users_min, left_on='source', right_on='id', how='left')

In [46]:
merge4['name Source'] = merge4['name Source'].combine_first(merge4['displayName'])
merge4['type Source'] = merge4['type Source'].combine_first(merge4['type'])
merge4['group Source'] = merge4['group Source'].combine_first(merge4['group'])

In [47]:
merge4 = merge4.drop(['id', 'displayName', 'type', 'group'], axis=1)

## Connect targetID with node names

### Hubs

In [49]:
merge5 = pd.merge(merge4, hubs_min, left_on='target', right_on='id', how='left')

In [51]:
merge5 = merge5.rename(columns={'displayName':'name Target', 'type':'type Target', 'group':'group Target'})

In [52]:
merge5 = merge5.drop(['id'], axis=1)

### Challenges

In [54]:
merge6 = pd.merge(merge5, challenges_min, left_on='target', right_on='id', how='left')

In [56]:
merge6['name Target'] = merge6['name Target'].combine_first(merge6['displayName'])
merge6['type Target'] = merge6['type Target'].combine_first(merge6['type'])
merge6['group Target'] = merge6['group Target'].combine_first(merge6['group'])

In [57]:
merge6 = merge6.drop(['id', 'displayName', 'type', 'group'], axis=1)

### Opportunities

In [59]:
merge7 = pd.merge(merge6, opportunities_min, left_on='target', right_on='id', how='left')

In [61]:
merge7['name Target'] = merge7['name Target'].combine_first(merge7['displayName'])
merge7['type Target'] = merge7['type Target'].combine_first(merge7['type'])
merge7['group Target'] = merge7['group Target'].combine_first(merge7['group'])

In [62]:
merge7 = merge7.drop(['id', 'displayName', 'type', 'group'], axis=1)

### Contributors

In [64]:
merge8 = pd.merge(merge7, users_min, left_on='target', right_on='id', how='left')

In [66]:
merge8 = merge8.drop(['id', 'displayName', 'type', 'group'], axis=1)

## Exporting dataframe to csv

In [67]:
merge8.to_csv('graphdata_pandas_2023.csv', index=False)

In [68]:
print('column available for the dataframe are:')
for i in merge8.columns:
    print(i)

column available for the dataframe are:
source
target
type Edge
group Edge
name Source
type Source
group Source
name Target
type Target
group Target


# Making predition tabel

In [69]:
data = pd.read_csv('graphdata_pandas_2023.csv')

type of the node goes next to the name, this makes it easier to see what type a node is

In [70]:
for i in range(len(data)):
    data['name Source'][i] = data['name Source'][i] + str(" ") + data['type Source'][i]
    data['name Target'][i] = data['name Target'][i] + str(" ") + data['type Target'][i]

In [71]:
data[data['name Source']==data['name Target']]

,source,target,type Edge,group Edge,name Source,type Source,group Source,name Target,type Target,group Target


There arent any source nodes and target nodes that refer to themselfes

This seems like a lot of different journeys and contributors, but every connection a user has is put on a different row, so if a user is a member of four different communities, it is counted as 4 different users.

In [75]:
data.to_csv('data_graph_matrix_2023.csv')